In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Uncomment the following lines to enabled CGroups verbose logging
#logging.getLogger('cgroups').setLevel(logging.DEBUG)
#logging.getLogger('cgroups.cpuset').setLevel(logging.DEBUG)

In [2]:
import json
import operator

import devlib
import trappy
import bart

from bart.sched.SchedMultiAssert import SchedMultiAssert
from wlgen import RTA, Periodic

## Global configuration

In [3]:
# Host side results folder
RESULTS_DIR = '/tmp/schedtest'

# Taerget side temporary folder
TARGET_DIR = '/root/schedtest'

# List of tools to install on the target system
TOOLS = ["rt-app", "trace-cmd", "taskset", "cgroup_run_into.sh"]

# List of modules to enable
MODULES = ['cgroups', 'bl']

## Target connection

In [4]:
from env import TestEnv

my_target_conf = {
    "platform"    : "linux",
    "board"       : "juno",
    "host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",
    "rtapp-calib" : {
        '0': 363, '1': 138, '2': 139, '3': 352, '4': 353, '5': 361
    },
}

# Setup the required Test Environment supports
my_tests_conf = {
    # list of additional devlib modules to install 
    "modules" : ['cgroups', 'bl', 'cpufreq'],
    # list of additional binary tools to install
    "tools" : ['rt-app', 'trace-cmd', 'cgroup_run_into.sh'],
    "ftrace" : {
         "events" : [
             "sched_switch"
         ],
         "buffsize" : 10240
    }
}

te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

# Report target connection
logging.info('Connected to %s target', target.abi)

06:38:54  INFO    :         Target - Using base path: /home/derkling/Code/lisa
06:38:54  INFO    :         Target - Loading custom (inline) target configuration
06:38:54  INFO    :         Target - Loading custom (inline) test configuration
06:38:54  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'cgroups', 'hwmon']
06:38:54  INFO    :         Target - Connecting linux target:
06:38:54  INFO    :         Target -   username : root
06:38:54  INFO    :         Target -       host : 192.168.0.1
06:38:54  INFO    :         Target -   password : 
06:39:44  INFO    :         Target - Initializing target workdir:
06:39:44  INFO    :         Target -    /root/devlib-target
06:39:53  INFO    :         Target - Topology:
06:39:53  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
06:39:55  INFO    :       Platform - Loading default EM:
06:39:55  INFO    :       Platform -    /home/derkling/Code/lisa/libs/utils/platforms/juno.json
06:39:57  INFO    :         FTrace - Enab

## List available Controllers

In [5]:
logging.info('%14s - Available controllers:', 'CGroup')
ssys = target.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    logging.info('%14s -    %10s (hierarchy id: %d) has %d cgroups',
                 'CGroup', n, h, g)

06:39:57  INFO    :         CGroup - Available controllers:
06:39:57  INFO    :         CGroup -        cpuset (hierarchy id: 1) has 2 cgroups
06:39:57  INFO    :         CGroup -           cpu (hierarchy id: 2) has 2 cgroups
06:39:57  INFO    :         CGroup -     schedtune (hierarchy id: 3) has 1 cgroups
06:39:57  INFO    :         CGroup -        memory (hierarchy id: 4) has 1 cgroups
06:39:57  INFO    :         CGroup -       devices (hierarchy id: 5) has 1 cgroups
06:39:57  INFO    :         CGroup -       freezer (hierarchy id: 6) has 1 cgroups
06:39:57  INFO    :         CGroup -    perf_event (hierarchy id: 7) has 1 cgroups
06:39:57  INFO    :         CGroup -       hugetlb (hierarchy id: 8) has 1 cgroups
06:39:57  INFO    :         CGroup -          pids (hierarchy id: 9) has 1 cgroups


## Example of CPUSET controller usage

In [6]:
# Get a reference to the CPUSet controller
cpuset = target.cgroups.controller('cpuset')

In [7]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
logging.info('Existing CGropups:')
for cg in cgroups:
    logging.info('  %s', cg)

06:39:58  INFO    : Existing CGropups:
06:39:58  INFO    :   /
06:39:58  INFO    :   /LITTLE


In [8]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    cpus = attrs['cpus']
    logging.info('%s:%-15s cpus: %s', cpuset.kind, cgroup.name, cpus)
    

06:39:58  INFO    : cpuset:/               cpus: 0-5
06:39:59  INFO    : cpuset:/LITTLE         cpus: 0,3-5


In [9]:
# Create a LITTLE partition
cpuset_littles = cpuset.cgroup('/LITTLE')

In [10]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "cpu_exclusive": "0", 
    "memory_spread_page": "0", 
    "sched_load_balance": "1", 
    "cpus": "0,3-5", 
    "effective_mems": "0", 
    "mem_hardwall": "0", 
    "mem_exclusive": "0", 
    "memory_pressure": "0", 
    "effective_cpus": "0,3-5", 
    "mems": "0", 
    "sched_relax_domain_level": "-1", 
    "memory_migrate": "0", 
    "memory_spread_slab": "0"
}


In [11]:
# Tune CPUs and MEMs attributes
#   they must be initialize for the group to be usable
cpuset_littles.set(cpus=target.bl.littles, mems=0)
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "cpu_exclusive": "0", 
    "memory_spread_page": "0", 
    "sched_load_balance": "1", 
    "cpus": "0,3-5", 
    "effective_mems": "0", 
    "mem_hardwall": "0", 
    "mem_exclusive": "0", 
    "memory_pressure": "0", 
    "effective_cpus": "0,3-5", 
    "mems": "0", 
    "sched_relax_domain_level": "-1", 
    "memory_migrate": "0", 
    "memory_spread_slab": "0"
}


In [12]:
# Define a periodic big (80%) task
task = Periodic(
    period_ms=100,
    duty_cycle_pct=80,
    duration_s=5).get()

# Create one task per each CPU in the target
tasks={}
for tid in enumerate(target.core_names):
    tasks['task{}'.format(tid[0])] = task

# Configure RTA to run all these tasks
rtapp = RTA(target, 'simple', calibration=te.calibration())
rtapp.conf(kind='profile', params=tasks, run_dir=TARGET_DIR);

06:40:01  INFO    :          WlGen - Setup new workload simple
06:40:01  INFO    :          RTApp - Workload duration defined by longest task
06:40:01  INFO    :          RTApp - Default policy: SCHED_OTHER
06:40:01  INFO    :          RTApp - ------------------------
06:40:01  INFO    :          RTApp - task [task0], sched: using default policy
06:40:01  INFO    :          RTApp -  | calibration CPU: 1
06:40:01  INFO    :          RTApp -  | loops count: 1
06:40:01  INFO    :          RTApp - + phase_000001: duration 5.000000 [s] (50 loops)
06:40:01  INFO    :          RTApp - |  period   100000 [us], duty_cycle  80 %
06:40:01  INFO    :          RTApp - |  run_time  80000 [us], sleep_time  20000 [us]
06:40:01  INFO    :          RTApp - ------------------------
06:40:01  INFO    :          RTApp - task [task1], sched: using default policy
06:40:01  INFO    :          RTApp -  | calibration CPU: 1
06:40:01  INFO    :          RTApp -  | loops count: 1
06:40:01  INFO    :          RTAp

In [13]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=te.ftrace, cgroup=cpuset_littles.name, out_dir=te.res_dir)

06:40:05  INFO    :          WlGen - Workload execution START:
06:40:05  INFO    :          WlGen -    /root/devlib-target/bin/cgroup_run_into.sh /LITTLE '/root/devlib-target/bin/rt-app /root/schedtest/simple_00.json'
06:42:13  INFO    :          WlGen - Pulling trace file into [/home/derkling/Code/lisa/results/20160225_183957/simple_00.dat]...


In [14]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)

In [15]:
# Compute and visualize tasks residencies on LITTLE clusterh CPUs
s = SchedMultiAssert(trappy.Run(trace), te.topology, execnames="task")
residencies = s.getResidency('cluster', target.bl.littles, percent=True)
print json.dumps(residencies, indent=4)

{
    "2798": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2799": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2800": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2801": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2802": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "2803": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }
}


/home/derkling/Code/lisa/libs/trappy/trappy/__init__.py:48: UserWarning: The Run object is deprecated.  Use trappy.FTrace instead
  warnings.warn("The Run object is deprecated.  Use trappy.FTrace instead")


In [16]:
# Assert that ALL tasks have always executed only on LITTLE cluster
s.assertResidency('cluster', target.bl.littles,
                  99.9, operator.ge, percent=True, rank=len(residencies))

True

## Example of CPU controller usage

In [17]:
# Get a reference to the CPU controller
cpu = target.cgroups.controller('cpu')

In [18]:
# Create a big partition on that CPUS
cpu_littles = cpu.cgroup('/LITTLE')

In [19]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "stat": "throttled_time 924635127680", 
    "cfs_period_us": "100000", 
    "shares": "1024", 
    "cfs_quota_us": "50000"
}


In [20]:
# Set a 1CPU equivalent bandwidth for that CGroup
cpu_littles.set(cfs_period_us=100000, cfs_quota_us=50000)
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "stat": "throttled_time 924635127680", 
    "cfs_period_us": "100000", 
    "shares": "1024", 
    "cfs_quota_us": "50000"
}


In [21]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=te.ftrace, cgroup=cpu_littles.name)

06:42:24  INFO    :          WlGen - Workload execution START:
06:42:24  INFO    :          WlGen -    /root/devlib-target/bin/cgroup_run_into.sh /LITTLE '/root/devlib-target/bin/rt-app /root/schedtest/simple_00.json'
06:44:32  INFO    :          WlGen - Pulling trace file into [.//simple_00.dat]...


In [22]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)